## 套件引入

In [ ]:
import bs4
import requests
import json

## 學期更改
* 更新資料改這邊就好

In [ ]:
currentYear = 109
currentSemester = 1

## 爬蟲程式碼
* 感謝 110 級大學長的程式碼
* 小弟在此邊上註解

In [12]:
# dataform 處理
data = '<tr><td+width=""33%"">開課學年：' + str(currentYear) + '　　開課學期：第' + str(currentSemester) + '學期</td><td+width=""33%"">開課部別：大學部</td><td+width=""34%"">開課系所：無</td></tr><tr><td+width=""33%"">開課班級：無</td><td+width=""33%"">授課教師：無</td><td+width=""34%"">上課時間：無</td></tr>'
req = requests.post('https://course.nuk.edu.tw/QueryCourse/QueryResult.asp', data = {'Condition':data,'Flag': 1,'OpenYear': currentYear,'Helf': currentSemester,'Pclass': 'A','Sclass': '','Yclass': '','SirName': '','Sirno': '','WeekDay': '','Card': '','Subject': '','Language': '','Pre_Cono': '','Coname': ''})
# 編碼問題 "big5"
req.encoding = 'big5'
soup = bs4.BeautifulSoup(req.text,'html.parser')
# <tr> 存取每 1 個課程的資料
soup = soup.find_all('tr')

# i 是要略過非課程的資料
i = 0

all_course = [] #總課表

for ele in soup:
    if i<=3:
        i+=1
        continue
    tdSet = ele.find_all('td')

    #微學分奇怪的格式 (判斷 : 學分未滿 1 但大於 0)
    if float(tdSet[6].text.upper()) < 1.0 and float(tdSet[6].text.upper()) > 0.0:
        continue    # 略過囉
    tempCourse = {}    # 存取課程的詳細資料[id, dept, name, grade, class, teacher, period, category, credit, location]
    tempCourse['id'] = tdSet[0].text.upper() + tdSet[1].text.upper()
    tempCourse['dept'] = tdSet[0].text.upper()
    tempCourse['name'] = tdSet[5].text
    tempCourse['grade'] = tdSet[3].text
    tempCourse['class'] = tdSet[4].text
    tempCourse['teacher'] = tdSet[12].decode_contents()[0:-5].replace('<br/>',',')
    
    # 課程時間處理 以 list 存放多個時段
    course_time = []
    tempCourseTime = []
    for i in range(14,21):
        if tdSet[i].text != '\u3000':    # 編碼問題
            for day in tdSet[i].text.split(','):
                if day == 'X':
                    course_time.append([i-13, 0])
                elif day == 'Y':
                    course_time.append([i-13, 4.5])   # 中午顯示 4.5
                else:
                    try:
                        course_time.append([i-13, int(day)])
                    except ValueError:
                        print("Convert to int error: {}".format(day))    # 印出錯誤格式
                        pass
    
    tempCourse['period'] = course_time
    # 必修以 True 表示, 選修以 False
    if tdSet[7].text == '必':
        tempCourse['category'] = True
    else:
        tempCourse['category'] = False
    tempCourse['credit'] = tdSet[6].text
    tempCourse['location'] = tdSet[13].text
    
    # 放入字典
    all_course.append(tempCourse)
    
# JSON存檔
with open('AllCourse.json','w',encoding='utf8') as f:
    f.write(json.dumps(all_course, ensure_ascii=False).encode("utf8",errors='ignore').decode("utf8",errors='ignore'))